In [1]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url, header=0)

In [2]:
df = tables[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
import numpy as np
df.replace('Not assigned', np.nan, inplace=True)

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
df['Neighbourhood'] = df['Neighbourhood'].mask(pd.isnull, df['Borough'])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Ignore cells with a borough that is Not assigned.

In [5]:
df.dropna(subset=['Borough'], axis=0, inplace=True)

#### Combine neighbourhoods with the same postcode.

In [6]:
df = df.groupby(['Postcode', 'Borough'], as_index=False, sort=False).agg({'Neighbourhood': lambda x: x.str.cat(sep=', ')})
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
df_latlng = pd.read_csv("http://cocl.us/Geospatial_data")
df_latlng.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [8]:
df_toronto = df.join(df_latlng.set_index('Postcode'), on='Postcode')
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Foursquare credentials are deleted for privacy concerns.

In [9]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

## Explore Downtown Toronto

In [10]:
df_downtown = df_toronto[df_toronto['Borough'] == 'Downtown Toronto']
df_downtown.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


#### function that processes all neighbourhoods in downtown toronto.

In [11]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighbourhood and create a new dataframe called *dt_venues*

In [12]:
dt_venues = getNearbyVenues(names=df_downtown['Neighbourhood'],
                            latitudes=df_downtown['Latitude'],
                            longitudes=df_downtown['Longitude']
                            )
dt_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [13]:
print('There are {} uniques categories.'.format(len(dt_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [14]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighbourhood'] = dt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighbourhood and calculate the mean of the frequency of occurrence of each category

In [15]:
dt_grouped = dt_onehot.groupby('Neighbourhood').mean().reset_index()
dt_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0300,0.0,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0100,0.0,0.0000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0000,0.0,0.0000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0000
3,"Cabbagetown, St. James Town",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0125,0.0,...,0.0,0.0,0.0,0.0,0.012500,0.0,0.0,0.0125,0.0,0.0125


In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = dt_grouped['Neighbourhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Bakery,Asian Restaurant,Burger Joint,Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Bakery,Café,Seafood Restaurant,Farmers Market,Cocktail Bar,Steakhouse,Cheese Shop,Beer Bar,Jazz Club
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Terminal
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Café,Pub,Chinese Restaurant,Italian Restaurant,Bakery,Pizza Place,Market
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Café,Spa,Bubble Tea Shop,Japanese Restaurant,Gym / Fitness Center


In [17]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from IPython.display import display

## Clustering neighbourhoods

In [18]:
# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighbourhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [19]:
dt_merged = df_downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dt_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Theater,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Brewery
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Fast Food Restaurant,Bakery,Sporting Goods Shop,Bookstore,Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Clothing Store,Diner,Bakery,Beer Bar,Cosmetics Shop
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Café,Seafood Restaurant,Farmers Market,Cocktail Bar,Steakhouse,Cheese Shop,Beer Bar,Jazz Club
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Café,Spa,Bubble Tea Shop,Japanese Restaurant,Gym / Fitness Center


In [20]:
dt_merged['Cluster Labels'].fillna(0, inplace=True)
dt_merged['Cluster Labels'] = dt_merged['Cluster Labels'].astype('int')

# create map
latitude = 43.6487
longitude = -79.38544
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

In [27]:
map_clusters

In [22]:
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Theater,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Brewery
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Fast Food Restaurant,Bakery,Sporting Goods Shop,Bookstore,Restaurant
15,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Clothing Store,Diner,Bakery,Beer Bar,Cosmetics Shop
20,Downtown Toronto,0,Coffee Shop,Bakery,Café,Seafood Restaurant,Farmers Market,Cocktail Bar,Steakhouse,Cheese Shop,Beer Bar,Jazz Club
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Café,Spa,Bubble Tea Shop,Japanese Restaurant,Gym / Fitness Center
30,Downtown Toronto,0,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Bakery,Asian Restaurant,Burger Joint,Restaurant,American Restaurant
36,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Restaurant,Fried Chicken Joint,Scenic Lookout,Italian Restaurant,Brewery,Music Venue
42,Downtown Toronto,0,Coffee Shop,Café,Hotel,American Restaurant,Bar,Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant,Steakhouse
48,Downtown Toronto,0,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Seafood Restaurant,Bakery,Steakhouse,Gym,Italian Restaurant
92,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Japanese Restaurant,Hotel,Seafood Restaurant,Beer Bar,Bakery,Italian Restaurant,Lounge


In [23]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [24]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Downtown Toronto,2,Airport Service,Airport Lounge,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Terminal


In [25]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,Downtown Toronto,3,Café,Restaurant,Italian Restaurant,Japanese Restaurant,Bookstore,Sandwich Place,Theater,Bakery,Bar,Pub
84,Downtown Toronto,3,Café,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Park,Dessert Shop


In [26]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,4,Café,Grocery Store,Park,Nightclub,Candy Store,Bank,Restaurant,Italian Restaurant,Baby Store,Diner
